# Wokflow  **TEST** zLightGBM - En Abril

## Inicializacion

In [5]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-14 17:36:12 UTC"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,697167,37.3,1454662,77.7,1454662,77.7
Vcells,1298503,10.0,8388608,64.0,1975146,15.1


In [6]:
source("/home/vickydiliscia/dmeyf2025/src/churn/C2010/config/_index.R")

In [7]:
source(file.path(PARAM$paths$functions, "index.R"))
source_functions(PARAM$paths$functions)

Sourcing: add_divisiones.R

Sourcing: add_ratio.R

Sourcing: agregar_clase_ternaria.R

Sourcing: agregar_var_rf.R

Sourcing: aplicar_dataquality_zeros.R

Sourcing: apply_sum_specs_multi.R

Sourcing: audit_nullear.R

Sourcing: build_variables_rotas.R

Sourcing: crear_proporcion.R

Sourcing: features_historicos_optimizados.R

Loading required package: Rcpp

Sourcing: features_historicos.R

Sourcing: interpolar_faltantes.R

Sourcing: lags.R

Sourcing: normalizar_ctrx_quarter.R

Sourcing: print_utils.R

Sourcing: run_features_historicas.R

Sourcing: tendencias_y_mas.R

Sourcing: ver_meses_rotos.R

✅ Todas las funciones fueron cargadas correctamente.



In [8]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Generacion de la clase_ternaria

In [9]:
Sys.time()
require( "data.table" )
dataset <- fread(PARAM$paths$datasetcrudo)

dataset <- agregar_clase_ternaria(dataset)

setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

[1] "2025-11-14 17:36:32 UTC"

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
201901,BAJA+1,645
201901,BAJA+2,729
201901,CONTINUA,122899
201902,BAJA+1,733
201902,BAJA+2,707
201902,CONTINUA,123961
201903,BAJA+1,708
201903,BAJA+2,751
201903,CONTINUA,124508


### Eliminacion de Features

In [10]:
# Salsa Magica para 202106
dataset[, mprestamos_personales := NULL ]
dataset[, cprestamos_personales := NULL ]

### Data Quality

In [11]:
variables_rotas <- build_variables_rotas(dataset)
# pretty_print_list(variables_rotas)

In [12]:
if (PARAM$dataquality$zeros == "nullear") {
  dataset <- aplicar_dataquality_zeros(
    dataset,
    variables_rotas,
    modo = "nullear"
  )

} else if (PARAM$dataquality$zeros == "interpolar") {
  dataset <- aplicar_dataquality_zeros(
    dataset,
    variables_rotas,
    modo = "interpolar",
    treat_zero_as_na = TRUE,
    require_both     = TRUE,
    only_if_na       = TRUE,
    integer_prefix   = "^c",
    round_integers   = TRUE
  )

} else {
  message("ℹ️ No se aplicó tratamiento de dataquality (PARAM$dataquality distinto de 'nullear' o 'interpolar').")
}


In [13]:
library(data.table)

# Auditor para verificar "nullear" aplicado según variables_rotas
# - Chequea que NO queden ceros en los meses marcados por variable
# - Resume NAs y ceros dentro/afuera
# - Mantiene el tipo de la columna (info)
audit_nullear <- function(dt, variables_rotas, month_col = "foto_mes",
                          replace_all_in_month = FALSE) {
  stopifnot(is.data.table(dt), month_col %in% names(dt))

  # helpers seguros para conteos
  count_zeros <- function(x) {
    if (is.numeric(x)) sum(x == 0, na.rm = TRUE) else NA_integer_
  }
  count_nas <- function(x) sum(is.na(x))

  res_list <- lapply(names(variables_rotas), function(v){
    if (!v %in% names(dt)) {
      return(data.table(
        variable = v, tipo = NA_character_,
        meses_marcados = NA_character_,
        filas_marcadas = NA_integer_,
        ceros_en_marcados = NA_integer_,
        NA_en_marcados = NA_integer_,
        ceros_fuera = NA_integer_,
        NA_fuera = NA_integer_,
        ok_sin_ceros_marc = NA, 
        modo = if (replace_all_in_month) "nullear_todo_mes" else "nullear_solo_ceros",
        status = "missing_column"
      ))
    }

    meses <- variables_rotas[[v]]
    if (!length(meses)) {
      return(data.table(
        variable = v, tipo = class(dt[[v]])[1],
        meses_marcados = "", filas_marcadas = 0L,
        ceros_en_marcados = 0L, NA_en_marcados = 0L,
        ceros_fuera = count_zeros(dt[[v]]),
        NA_fuera = count_nas(dt[[v]]),
        ok_sin_ceros_marc = TRUE,
        modo = if (replace_all_in_month) "nullear_todo_mes" else "nullear_solo_ceros",
        status = "no_months"
      ))
    }

    dentro <- dt[get(month_col) %in% meses, ..v][[1]]
    fuera  <- dt[!get(month_col) %in% meses, ..v][[1]]
    cls    <- class(dt[[v]])[1]

    zeros_in  <- count_zeros(dentro)
    nas_in    <- count_nas(dentro)
    zeros_out <- count_zeros(fuera)
    nas_out   <- count_nas(fuera)

    data.table(
      variable = as.character(v),
      tipo = cls,
      meses_marcados = paste(sort(unique(meses)), collapse = ","),
      filas_marcadas = length(dentro),
      ceros_en_marcados = zeros_in,
      NA_en_marcados = nas_in,
      ceros_fuera = zeros_out,
      NA_fuera = nas_out,
      ok_sin_ceros_marc = is.na(zeros_in) || zeros_in == 0L,
      modo = if (replace_all_in_month) "nullear_todo_mes" else "nullear_solo_ceros",
      status = "ok"
    )
  })

  out <- rbindlist(res_list, fill = TRUE)

  # Orden: primero las problemáticas (ok_sin_ceros_marc == FALSE), luego alfabético
  out[, bad := fifelse(is.na(ok_sin_ceros_marc), TRUE, !ok_sin_ceros_marc)]
  setorder(out, -bad, variable)
  out[, bad := NULL][]
}

# Ejemplo de uso:
# res <- audit_nullear(dataset, variables_rotas, month_col = "foto_mes", replace_all_in_month = FALSE)
# res[!ok_sin_ceros_marc | status != "ok"]
# stopifnot(all(res$ok_sin_ceros_marc[res$status == "ok"]))



In [14]:
# Si usaste replace_all_in_month = FALSE (nullear solo ceros):
res <- audit_nullear(dataset, variables_rotas, month_col = "foto_mes",
                     replace_all_in_month = FALSE)

# Ver primeras filas y variables con problemas (si las hubiera)
res[ok_sin_ceros_marc | status == "ok"][]


variable,tipo,meses_marcados,filas_marcadas,ceros_en_marcados,NA_en_marcados,ceros_fuera,NA_fuera,ok_sin_ceros_marc,modo,status
<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<lgl>,<chr>,<chr>
active_quarter,integer,202006,153102,0,153102,64636,0,TRUE,nullear_solo_ceros,ok
catm_trx,integer,202006,153102,0,153102,2600186,0,TRUE,nullear_solo_ceros,ok
catm_trx_other,integer,202006,153102,0,153102,3424375,0,TRUE,nullear_solo_ceros,ok
ccajas_consultas,integer,202006,153102,0,153102,4234723,0,TRUE,nullear_solo_ceros,ok
ccajas_depositos,integer,"202006,202105",317019,0,317019,4115433,0,TRUE,nullear_solo_ceros,ok
ccajas_extracciones,integer,202006,153102,0,153102,4046193,0,TRUE,nullear_solo_ceros,ok
ccajas_otras,integer,202006,153102,0,153102,4429487,0,TRUE,nullear_solo_ceros,ok
ccajas_transacciones,integer,202006,153102,0,153102,3749353,0,TRUE,nullear_solo_ceros,ok
ccajeros_propios_descuentos,integer,"201910,202002,202006,202009,202010,202102",914901,0,914901,3631950,0,TRUE,nullear_solo_ceros,ok


### Data Drifting

Se debe corregir el drifting natural que ocurre en loa datos, en particular los datos monetarios que se vieron fuertemente afectados por una alta inflación
<br> Posibles métodos son:
* No hacer absolutamente nada
* Ajuste de valores monetarios por indices del tipo :
   * IPC  Indice de Precios al Consumidor
   * Dolar Oficial
   * Dolar Blue
   * UVA  Unidad de Valor Adquisitivo

a este codigo de Data Drifting lo debera escribir usted

### Feature Engineering Intra-Mes

In [15]:
if (isTRUE(PARAM$FE_intrames$kmes)) {
  dataset[, kmes := foto_mes %% 100]
  message("Columna kmes creada")
} else {
  message("Columna kmes no creada")
}

if (isTRUE(PARAM$FE_intrames$normquarter)) {
  dataset <- normalizar_ctrx_quarter(dataset)
  message("Normalizacion ctrx_quarter aplicada")
} else {
  message("Normalizacion ctrx_quarter no aplicada")
}

# variable extraida de una tesis de maestria de Irlanda, se perdió el link
# dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

Sys.time()

Columna kmes creada

Normalizacion ctrx_quarter aplicada



[1] "2025-11-14 17:37:42 UTC"

In [16]:
spec_sumas <- fread(PARAM$FE_intrames$specsumas)

# Normalizar nombres de columnas
setnames(spec_sumas, tolower(names(spec_sumas)))
req <- c("new_var","inputs","active")
faltan <- setdiff(req, names(spec_sumas))
if (length(faltan)) stop("Faltan columnas en spec: ", paste(faltan, collapse=", "))

# Limpiar espacios en inputs y new_var
spec_sumas[, new_var := trimws(new_var)]
spec_sumas[, inputs  := trimws(inputs)]

# 2) Convertir 'active' a lógico robusto
to_bool <- function(x){
  if (is.logical(x)) return(x)
  x2 <- tolower(trimws(as.character(x)))
  x2 %in% c("true","t","1","yes","y","si","sí")
}
spec_sumas[, active := to_bool(active)]

dataset <- apply_sum_specs_multi(
  dataset,
  spec = spec_sumas,
  from_file = FALSE,
  sep_inputs = ","
)



✅ 'mtarjeta_total_consumo' = suma_segura(mtarjeta_visa_consumo+mtarjeta_master_consumo)

✅ 'mtarjeta_total_debitos_automaticos' = suma_segura(mttarjeta_visa_debitos_automaticos+mttarjeta_master_debitos_automaticos)

✅ 'mfinanciacionlimite_total' = suma_segura(Master_mfinanciacion_limite+Visa_mfinanciacion_limite)

✅ 'msaldopesos_total' = suma_segura(Master_msaldopesos+Visa_msaldopesos)

✅ 'msaldodolares_total' = suma_segura(Master_msaldodolares+Visa_msaldodolares)

✅ 'mconsumopesos_total' = suma_segura(Master_mconsumospesos+Visa_mconsumospesos)

✅ 'mconsumodolares_total' = suma_segura(Master_mconsumosdolares+Visa_mconsumosdolares)

✅ 'mlimitecompra_total' = suma_segura(Master_mlimitecompra+Visa_mlimitecompra)

✅ 'madelantopesos_total' = suma_segura(Master_madelantopesos+Visa_madelantopesos)

✅ 'madelantodolares_total' = suma_segura(Master_madelantodolares+Visa_madelantodolares)

✅ 'mpagado_total' = suma_segura(Master_mpagado+Visa_mpagado)

✅ 'mpagopesos_total' = suma_segura(Master_mpag

### Feature Engineering Historico

In [17]:
# 1) ¿Es data.table?
is.data.table(dataset)

# 2) ¿Qué columnas está intentando laguear?
id_col <- "numero_de_cliente"; time_col <- "foto_mes"
exclude <- c("clase_ternaria")

cols_lagueables <- setdiff(names(dataset), c(id_col, time_col, exclude))
length(cols_lagueables); head(cols_lagueables)

# 3) ¿Hay más de un registro por cliente (si no, los lags serían todo NA)?
dataset[, .N, by = id_col][N > 1L][1:5]


[1] TRUE

[1] 174

[1] "active_quarter"     "cliente_vip"        "internet"          
[4] "cliente_edad"       "cliente_antiguedad" "mrentabilidad"

numero_de_cliente,N
<int>,<int>
303233007,24
503840997,16
511125603,27
609024937,24
619465394,17


In [18]:
setorder(dataset, numero_de_cliente, foto_mes)

if (isTRUE(PARAM$FE_hist$Lags$Run)) {
  dataset <- crear_lags(dataset, lags = PARAM$FE_hist$Lags$Order)
} else {
  message("Lags no aplicados")
}

if (isTRUE(PARAM$FE_hist$DeltaLags$Run)) {
  dataset <- crear_delta_lags(dataset, lags = PARAM$FE_hist$DeltaLags$Order)
} else {
  message("Delta Lags no aplicados")
}

Sys.time()

Creando lags: 1, 2

✅ Lags creados: 348

Creando delta-lags: 1, 2

✅ Delta-lags creados: 348



[1] "2025-11-14 17:44:28 UTC"

In [19]:
colnames(dataset)

[1] "numero_de_cliente"                          
  [2] "foto_mes"                                   
  [3] "active_quarter"                             
  [4] "cliente_vip"                                
  [5] "internet"                                   
  [6] "cliente_edad"                               
  [7] "cliente_antiguedad"                         
  [8] "mrentabilidad"                              
  [9] "mrentabilidad_annual"                       
 [10] "mcomisiones"                                
 [11] "mactivos_margen"                            
 [12] "mpasivos_margen"                            
 [13] "cproductos"                                 
 [14] "tcuentas"                                   
 [15] "ccuenta_corriente"                          
 [16] "mcuenta_corriente_adicional"                
 [17] "mcuenta_corriente"                          
 [18] "ccaja_ahorro"                               
 [19] "mcaja_ahorro"                               
 [20] "mcaja_ahorro_adicional"                     
 [21] "mcaja_ahorro_dolares"                       
 [22] "cdescubierto_preacordado"                   
 [23] "mcuentas_saldo"                             
 [24] "ctarjeta_debito"                            
 [25] "ctarjeta_debito_transacciones"              
 [26] "mautoservicio"                              
 [27] "ctarjeta_visa"                              
 [28] "ctarjeta_visa_transacciones"                
 [29] "mtarjeta_visa_consumo"                      
 [30] "ctarjeta_master"                            
 [31] "ctarjeta_master_transacciones"              
 [32] "mtarjeta_master_consumo"                    
 [33] "cprestamos_prendarios"                      
 [34] "mprestamos_prendarios"                      
 [35] "cprestamos_hipotecarios"                    
 [36] "mprestamos_hipotecarios"                    
 [37] "cplazo_fijo"                                
 [38] "mplazo_fijo_dolares"                        
 [39] "mplazo_fijo_pesos"                          
 [40] "cinversion1"                                
 [41] "minversion1_pesos"                          
 [42] "minversion1_dolares"                        
 [43] "cinversion2"                                
 [44] "minversion2"                                
 [45] "cseguro_vida"                               
 [46] "cseguro_auto"                               
 [47] "cseguro_vivienda"                           
 [48] "cseguro_accidentes_personales"              
 [49] "ccaja_seguridad"                            
 [50] "cpayroll_trx"                               
 [51] "mpayroll"                                   
 [52] "mpayroll2"                                  
 [53] "cpayroll2_trx"                              
 [54] "ccuenta_debitos_automaticos"                
 [55] "mcuenta_debitos_automaticos"                
 [56] "ctarjeta_visa_debitos_automaticos"          
 [57] "mttarjeta_visa_debitos_automaticos"         
 [58] "ctarjeta_master_debitos_automaticos"        
 [59] "mttarjeta_master_debitos_automaticos"       
 [60] "cpagodeservicios"                           
 [61] "mpagodeservicios"                           
 [62] "cpagomiscuentas"                            
 [63] "mpagomiscuentas"                            
 [64] "ccajeros_propios_descuentos"                
 [65] "mcajeros_propios_descuentos"                
 [66] "ctarjeta_visa_descuentos"                   
 [67] "mtarjeta_visa_descuentos"                   
 [68] "ctarjeta_master_descuentos"                 
 [69] "mtarjeta_master_descuentos"                 
 [70] "ccomisiones_mantenimiento"                  
 [71] "mcomisiones_mantenimiento"                  
 [72] "ccomisiones_otras"                          
 [73] "mcomisiones_otras"                          
 [74] "cforex"                                     
 [75] "cforex_buy"                                 
 [76] "mforex_buy"                                 
 [77] "cforex_sell"                               

In [31]:
# aqui se agregan las tendencias de los ultimos 6 meses

cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    tendencias_y_mas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}

ncol(dataset)
Sys.time()

[1] 1047

[1] "2025-11-14 17:49:40 UTC"

### Variables adicionales historicas

In [32]:
spec_hist <- fread(PARAM$FE_hist$spechistoricas) # maneja bien las comillas por defecto

# dt es tu data.table con numero_de_cliente, foto_mes, etc.
apply_rolling_with_kernel(dataset, spec_hist)


Kernel Rcpp -> Var: mcuenta_corriente | win: 6 | ops: mean, sd, cv, prop_gt0, ratioavg

Kernel Rcpp -> Var: mcaja_ahorro | win: 6 | ops: mean, sd, cv, prop_gt0, ratioavg

Kernel Rcpp -> Var: mcuentas_saldo | win: 6 | ops: mean, sd, cv, ratioavg

Kernel Rcpp -> Var: mautoservicio | win: 6 | ops: mean, sd, cv, prop_gt0, ratioavg

Kernel Rcpp -> Var: mextraccion_autoservicio | win: 6 | ops: mean, sd, cv, prop_gt0, ratioavg

Kernel Rcpp -> Var: mtarjeta_visa_consumo | win: 6 | ops: mean, sd, cv, prop_gt0, ratioavg

Kernel Rcpp -> Var: mtarjeta_master_consumo | win: 6 | ops: mean, sd, cv, prop_gt0, ratioavg

Kernel Rcpp -> Var: mpagado_total | win: 6 | ops: mean, sd, cv, prop_gt0, ratioavg

Kernel Rcpp -> Var: mconsumototal_total | win: 6 | ops: mean, sd, cv, prop_gt0, ratioavg

Kernel Rcpp -> Var: msaldototal_total | win: 6 | ops: mean, sd, cv, ratioavg

Kernel Rcpp -> Var: mpayroll_total | win: 6 | ops: mean, sd, cv, prop_gt0, ratioavg

Kernel Rcpp -> Var: mpagodeservicios | win: 6 | ops:

### Feature Engineering a partir de hojas de Random Forest

In [33]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

Loading required package: lightgbm

Registered S3 methods overwritten by 'lightgbm':
  method                 from     
  dimnames<-.lgb.Dataset zlightgbm
  dim.lgb.Dataset        zlightgbm
  dimnames.lgb.Dataset   zlightgbm
  predict.lgb.Booster    zlightgbm
  print.lgb.Booster      zlightgbm
  summary.lgb.Booster    zlightgbm


Attaching package: ‘lightgbm’


The following objects are masked from ‘package:zlightgbm’:

    get_field, getLGBMthreads, lgb.configure_fast_predict,
    lgb.convert_with_rules, lgb.cv, lgb.Dataset, lgb.Dataset.construct,
    lgb.Dataset.create.valid, lgb.Dataset.save,
    lgb.Dataset.set.categorical, lgb.Dataset.set.reference,
    lgb.drop_serialized, lgb.dump, lgb.get.eval.result, lgb.importance,
    lgb.interprete, lgb.load, lgb.make_serializable, lgb.model.dt.tree,
    lgb.plot.importance, lgb.plot.interpretation, lgb.restore_handle,
    lgb.save, lgb.slice.Dataset, lgb.train, lightgbm, set_field,
    setLGBMthreads




In [34]:
# Feature Engineering agregando variables de Random Forest
#  aqui es donde se hace el trabajo
agregar_var_rf(PARAM)

Sys.time()

inicio AgregaVarRandomForest()
Fin construccion RandomForest
periodo =  201901 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201902 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201903 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201904 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201905 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201906 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201907 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  201908 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7

[1] "2025-11-14 18:06:15 UTC"

In [35]:
ncol(dataset)
colnames(dataset)

[1] 1488

[1] "numero_de_cliente"                          
   [2] "foto_mes"                                   
   [3] "active_quarter"                             
   [4] "cliente_vip"                                
   [5] "internet"                                   
   [6] "cliente_edad"                               
   [7] "cliente_antiguedad"                         
   [8] "mrentabilidad"                              
   [9] "mrentabilidad_annual"                       
  [10] "mcomisiones"                                
  [11] "mactivos_margen"                            
  [12] "mpasivos_margen"                            
  [13] "cproductos"                                 
  [14] "tcuentas"                                   
  [15] "ccuenta_corriente"                          
  [16] "mcuenta_corriente_adicional"                
  [17] "mcuenta_corriente"                          
  [18] "ccaja_ahorro"                               
  [19] "mcaja_ahorro"                               
  [20] "mcaja_ahorro_adicional"                     
  [21] "mcaja_ahorro_dolares"                       
  [22] "cdescubierto_preacordado"                   
  [23] "mcuentas_saldo"                             
  [24] "ctarjeta_debito"                            
  [25] "ctarjeta_debito_transacciones"              
  [26] "mautoservicio"                              
  [27] "ctarjeta_visa"                              
  [28] "ctarjeta_visa_transacciones"                
  [29] "mtarjeta_visa_consumo"                      
  [30] "ctarjeta_master"                            
  [31] "ctarjeta_master_transacciones"              
  [32] "mtarjeta_master_consumo"                    
  [33] "cprestamos_prendarios"                      
  [34] "mprestamos_prendarios"                      
  [35] "cprestamos_hipotecarios"                    
  [36] "mprestamos_hipotecarios"                    
  [37] "cplazo_fijo"                                
  [38] "mplazo_fijo_dolares"                        
  [39] "mplazo_fijo_pesos"                          
  [40] "cinversion1"                                
  [41] "minversion1_pesos"                          
  [42] "minversion1_dolares"                        
  [43] "cinversion2"                                
  [44] "minversion2"                                
  [45] "cseguro_vida"                               
  [46] "cseguro_auto"                               
  [47] "cseguro_vivienda"                           
  [48] "cseguro_accidentes_personales"              
  [49] "ccaja_seguridad"                            
  [50] "cpayroll_trx"                               
  [51] "mpayroll"                                   
  [52] "mpayroll2"                                  
  [53] "cpayroll2_trx"                              
  [54] "ccuenta_debitos_automaticos"                
  [55] "mcuenta_debitos_automaticos"                
  [56] "ctarjeta_visa_debitos_automaticos"          
  [57] "mttarjeta_visa_debitos_automaticos"         
  [58] "ctarjeta_master_debitos_automaticos"        
  [59] "mttarjeta_master_debitos_automaticos"       
  [60] "cpagodeservicios"                           
  [61] "mpagodeservicios"                           
  [62] "cpagomiscuentas"                            
  [63] "mpagomiscuentas"                            
  [64] "ccajeros_propios_descuentos"                
  [65] "mcajeros_propios_descuentos"                
  [66] "ctarjeta_visa_descuentos"                   
  [67] "mtarjeta_visa_descuentos"                   
  [68] "ctarjeta_master_descuentos"                 
  [69] "mtarjeta_master_descuentos"                 
  [70] "ccomisiones_mantenimiento"                  
  [71] "mcomisiones_mantenimiento"                  
  [72] "ccomisiones_otras"                          
  [73] "mcomisiones_otras"                          
  [74] "cforex"                                     
  [75] "cforex_buy"                                 
  [76] "mforex_buy"         

In [ ]:
guardar_db <- function(dataset, path, compress = "xz") {
  # Validaciones básicas
  if (missing(dataset)) stop("Debes pasar un dataset.")
  if (missing(path))    stop("Debes especificar un path para guardar el archivo.")
  
  # Asegurar extensión .rds si no la pusiste
  if (!grepl("\\.rds$", path, ignore.case = TRUE)) {
    path <- paste0(path, ".rds")
  }
  
  # Crear carpeta si no existe
  dir.create(dirname(path), recursive = TRUE, showWarnings = FALSE)
  
  # Guardar el dataset comprimido
  saveRDS(dataset, file = path, compress = compress)
  
  message(sprintf("✔ Dataset guardado en: %s", normalizePath(path)))
  invisible(path)
}


In [ ]:
# guardar_db(dataset, "/home/vickydiliscia/buckets/b1/datasets/dataset_churn_1")

### Optimizacion de Hipeparámetros

## Modelado

### Target Engineering

In [19]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)]

### Training Strategy

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202108  obviamente completo

* final_train =  [201901, 202106] sin undersampling de los CONTINUA

* training
   * testing = 202106
   * training = [201901, 202104]  donde se consideran el 5% de los CONTINUA

In [20]:
PARAM$trainingstrategy$testing <- c(202104)

PARAM$trainingstrategy$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102
)

PARAM$trainingstrategy$undersampling <- 0.05


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [21]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

#### Registros  cambio las proporciones de POS/NEG

In [22]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar := runif(nrow(dataset))]
dataset[, training := 0L]

dataset[  foto_mes %in%  PARAM$trainingstrategy$training &
  (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]


### Optimizacion de Hipeparámetros

Se optimizan los hiperparámetros maximizando la ganancia.

In [23]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

In [24]:
dtrain <- lgb.Dataset(
  data= data.matrix(dataset[training == 1L, campos_buenos, with = FALSE]),
  label= dataset[training == 1L, clase01],
  free_raw_data= TRUE
)

cat("filas", nrow(dtrain), "columnas", ncol(dtrain), "\n")
Sys.time()

filas 215803 columnas 1241 


[1] "2025-11-07 21:08:42 EST"

In [25]:
# defino los datos de testing
dataset_test <- dataset[foto_mes %in% PARAM$trainingstrategy$testing]

# precalculo el campo de la ganancia
dataset_test[, gan := -20000.0 ]
dataset_test[ clase_ternaria=="BAJA+2", gan := 780000]

# precalculo la test_matrix
test_matrix <- data.matrix(dataset_test[, campos_buenos, with= FALSE])

cat("filas", nrow(dataset_test), "columnas", ncol(dataset_test), "\n")
Sys.time()

filas 163418 columnas 1246 


[1] "2025-11-07 21:08:45 EST"

In [26]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers

Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames




In [27]:
# Especificacion de la cantidad de iteraciones de la Bayesian Optimization
# 50 es razonable
PARAM$hipeparametertuning$BO_iteraciones <- 30 # un 50 seria mas razonable

In [28]:
# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "custom",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  extra_trees = FALSE,

  max_depth = -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split = 0.0, # min_gain_to_split >= 0.0
  min_sum_hessian_in_leaf = 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1 = 0.0, # lambda_l1 >= 0.0
  lambda_l2 = 0.0, # lambda_l2 >= 0.0

  bagging_fraction = 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction = 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction = 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance = FALSE, #
  scale_pos_weight = 1.0, # scale_pos_weight > 0.0

  drop_rate = 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop = 50, # <=0 means no limit
  skip_drop = 0.5, # 0.0 <= skip_drop <= 1.0

  max_bin= 31
)

In [29]:
# Notar que se recorren algunos hiperparametros en forma logaritmica
#   y que con forbidden se tiene en cuenta el juego que hay entre min_data_in_leaf y num_leaves

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeNumericParam("num_iterations", lower= 0.0, upper= 11.1, trafo= function(x) as.integer( round(2^x)) ),
  makeNumericParam("learning_rate", lower= -8.0, upper= -1.0, trafo= function(x) 2^x ),
  makeNumericParam("feature_fraction", lower= 0.05, upper= 1.0 ),
  makeNumericParam("min_data_in_leaf", lower= 0.0, upper= log2(nrow(dtrain)/2), trafo= function(x) as.integer(round(2^x)) ),
  makeNumericParam("num_leaves", lower= 1.0, upper= 10.0, trafo= function(x) as.integer(round(2^x)) ),
  forbidden= quote( (2^min_data_in_leaf)*(2^num_leaves) > nrow(dtrain) )
)

El parámetro **ksemillerio** indica se se hace semillerio DENTRO de la bayesiana
* 1 **no** se hace Ensemble Semillerio, apenas se corre un solo LightGBM
* mayor a 1, se hace un  k-Ensemble Semillerio

El parámetro  **repe** indica si dentro de la bayesiana se toman varias medidas y luego se promedian
<br> Esto se hace ya sea que se llama a un solo LightGBM o se haceun Ensemble Semillerio de LightGBMs
<br> Tener en cuenta que repe multiplica linealmente el tiempo de corrida de la Bayesian Optimization

In [30]:
PARAM$hipeparametertuning$ksemillerio <- 1L
PARAM$hipeparametertuning$repe <- 1L


In [31]:
if(!require("primes")) install.packages("primes")
require("primes")

Loading required package: primes



In [32]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
# me quedo con PARAM$semillerio  primos al azar
PARAM$BO$semillas <- sample(primos)[seq( PARAM$hipeparametertuning$ksemillerio * PARAM$hipeparametertuning$repe )]

cat( PARAM$BO$semillas)

974411

In [33]:
if(!require("rlist")) install.packages("rlist")
require("rlist")

Loading required package: rlist



In [34]:
# logueo al archivo BO_log.txt
loguear  <- function( reg, arch=NA, verbose=TRUE )
{
  t0 <- Sys.time()
  archivo <- arch
  if( is.na(arch) ) archivo <- paste0( folder, substitute( reg), ext )


  if( !file.exists( archivo ) )
  {
    # Escribo los titulos
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  # escribo el registro
  linea  <- paste0( format(t0, "%Y%m%d.%H%M%S"),  "\t",     # la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  # grabo al archivo

  if( verbose )  cat( linea )   # imprimo por pantalla
}

In [35]:
# esto esta en una funcion para que el garbage collector lo libere
# entrena, aplica el modelo a testing, y devuelve el vector de probabilidades

OneTrainPredict <- function(param_completo) {
    
  modelo <- lgb.train(
    data= dtrain,
    param= param_completo
  )
  gmodelo <<- modelo

  # aplico el modelo a los datos nuevos
  pred <- predict(
    modelo,
    test_matrix
  )

  return( pred )
}

In [36]:
# En el argumento x llegan los parametros de la bayesiana
#  devuelve la ganancia en datos de testing

# aqui se ira guardando la mejor iteracion de la bayesiana
gmejor <- list()
gmejor$iter <- 0
gmejor$gan <- -Inf

giter <- 0
if( file.exists("BO_log.txt") ){
  tb_BO <- fread("BO_log.txt")
  giter <- nrow(tb_BO) -1 
}

EstimarGanancia_lightgbm <- function(x) {

  giter <<- giter + 1
  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  vgan_mesetas <- c()  # las ganancias, tengo repe de ellas

  # loop de las repeticionies
  for( repe in seq( PARAM$hipeparametertuning$repe ) )
  {
     desde <- (PARAM$hipeparametertuning$repe-1)*PARAM$hipeparametertuning$ksemillerio + 1
     hasta <- desde + PARAM$hipeparametertuning$ksemillerio -1
     rsemillas <- PARAM$BO$semillas[ desde:hasta ]

     # vector inicial de probabilidades
     vpred_acum <- rep( 0.0, nrow(dataset_test) )

     # loop del semillerio
     for( sem in rsemillas ) # itero semillerio
     {
        param_completo$seed <- sem  # asigno se semilla
        vpred_acum <- vpred_acum + OneTrainPredict( param_completo )
        
        gc(full= TRUE, verbose= FALSE)
     }

     # Calculo de ganancia suavizada de la meseta
     tb_prediccion <- dataset_test[, list(gan)]
     tb_prediccion[, prob := vpred_acum ]

     setorder(tb_prediccion, -prob)
     tb_prediccion[, gan_acum := cumsum(gan)]

     # la meseta es un punto, mil para la izquierda, otros mil para la derecha
     tb_prediccion[, gan_meseta :=
       frollmean(
         x= gan_acum, n= 2001, align= "center",
         na.rm= TRUE, hasNA= TRUE
      )
     ]

     vgan_mesetas <- c(vgan_mesetas, tb_prediccion[, max(gan_meseta, na.rm = TRUE)] )
  }

  gan_mesetas_prom <- mean( vgan_mesetas ) 

  if( gan_mesetas_prom > gmejor$gan ){
    gmejor$gan <<- gan_mesetas_prom
    gmejor$iter <<- giter

    # hrabo importancia de variables
    fwrite( lgb.importance(gmodelo),
      file= paste0("impo_", giter, ".txt"),
      sep= "\t"
    )
  }

  # datos qeu voy a loguear
  xx <- copy(param_completo)
  xx$iter <- giter
  xx$metrica_mejor <- gmejor$gan
  xx$metrica_sd <- sd(vgan_mesetas)
  xx$metrica <- gan_mesetas_prom

  loguear( xx, "BO_log.txt")
  set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")  # le reordeno a mlrMBO

  return( gan_mesetas_prom ) #tiempo_corrida) )
}

In [37]:
# Aqui comienza la configuracion de la Bayesian Optimization
#  es compleja la configuracion de una Bayesian Optimization

# en este archivo quedan la evolucion binaria de la BO
kbayesiana <- "bayesiana.RDATA"

funcion_optimizar <- EstimarGanancia_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output= FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo

obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar, # la funcion que voy a maximizar
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set= PARAM$hipeparametertuning$hs, # definido al comienzo del programa
  has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time= 600, # se graba cada 600 segundos
  save.file.path= kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hipeparametertuning$BO_iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)

Sys.time()

[1] "2025-11-07 21:08:46 EST"

#### Corrida de la Bayesian Optimization

In [38]:
# inicio la optimizacion bayesiana, retomando si ya existe
# es la celda mas lenta de todo el notebook

if (!file.exists(kbayesiana)) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

Sys.time()

Warning message in mboContinue(kbayesiana):
“Tuning ended with term.iter. No need to continue. Simply returning stored result.”


[1] "2025-11-07 21:08:46 EST"

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Obviamente los datos donde se aplica el modelo es el mes  {202108} , que no tiene clase
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los treinta meses del intervalo [201901, 202106]
* Se realiza undersampling al 10%
* Se utilizan los hiperparámetros optimos encontrados en la Bayesian Optimization
   * Se escala min_data_in_leaf

* Por experimentos en meses anteriores, se decide cortar en los 11000 registros con mayor probabildiad de POS={"BAJA+1", "BAJA+2"}, , esta es una *enorme* decisión.

### Final Training Strategy

In [39]:
PARAM$train_final$future <- c(202106)

PARAM$train_final$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104
)

PARAM$train_final$undersampling <- 0.10

In [40]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

#### Registros cambio las proporciones de POS/NEG

In [41]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [42]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Adaptacion Hiperparametros Optimos

Solamente escalo min_data_in_leaf  por  nrow(dataset_train_final) / nrow(dtrain)

In [43]:
# leo el archivo donde quedaron los hiperparametros optimos
tb_BO <-  fread("BO_log.txt")
setorder( tb_BO, -metrica)  # ordeno por metrica descendente
tb_BO[1]

fecha,objective,metric,first_metric_only,boost_from_average,feature_pre_filter,verbosity,force_row_wise,seed,extra_trees,⋯,max_bin,num_iterations,learning_rate,feature_fraction,min_data_in_leaf,num_leaves,iter,metrica_mejor,metrica_sd,metrica
<dbl>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<int>,<lgl>,<int>,<lgl>,⋯,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<lgl>,<dbl>
20251107,binary,custom,TRUE,TRUE,FALSE,-100,TRUE,974411,FALSE,⋯,31,2185,0.03765917,0.3817213,167,217,40,364699510,NA,364699510


In [44]:
# en la tabla ademas de los parametros del LightGBM, hay campos de salida
param_lgbm <- union( names(PARAM$lgbm$param_fijos),  names(PARAM$hipeparametertuning$hs$pars) )

PARAM$train_final$param_mejores <- as.list( tb_BO[1, param_lgbm, with=FALSE])

PARAM$train_final$param_mejores$min_data_in_leaf <- as.integer( round(PARAM$train_final$param_mejores$min_data_in_leaf * nrow(dataset_train_final[training == 1L]) / nrow(dtrain)))

cat( tb_BO[1, min_data_in_leaf] , PARAM$train_final$param_mejores$min_data_in_leaf, "\n")
PARAM$train_final$param_mejores

167 338 


$objective
[1] "binary"

$metric
[1] "custom"

$first_metric_only
[1] TRUE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$verbosity
[1] -100

$force_row_wise
[1] TRUE

$seed
[1] 974411

$extra_trees
[1] FALSE

$max_depth
[1] -1

$min_gain_to_split
[1] 0

$min_sum_hessian_in_leaf
[1] 0.001

$lambda_l1
[1] 0

$lambda_l2
[1] 0

$bagging_fraction
[1] 1

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] FALSE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$max_bin
[1] 31

$num_iterations
[1] 2185

$learning_rate
[1] 0.03765917

$feature_fraction
[1] 0.3817213

$min_data_in_leaf
[1] 338

$num_leaves
[1] 217

### Final Models

Aqui SIEMPRE voy a hacer un semillerio, independientemente de si en la Bayesian Optimization calculé un semillerio en cada iteración.
<br> Entreno un LightGBM para cada semilla,  y guardo el modelo dentro de la carpeta  **modelitos**
<br> Intencionalmente en una primera etapá se generan los modelos y graban, y en una segunda etapa se leen eso modelos y se aplican a los datos del futuro

APO controla cuantas veces se repite el modelo, que se usa para promediar ganancias y reportar en la Pseudo Competencia algo razonable
<br> El modelo puede ser un LightGBM simple (ksemillerio==1)  o un Ensemble Semillerio( ksemillerio > 1 )
<br> Lamentablmente APO necesita utilizar muchas semillas, y eso demanda TIEMPO de corrida

In [45]:
# Semillerio Final
PARAM$train_final$APO <- 10
PARAM$train_final$ksemillerio  <- 10

PARAM$train_final$cortes <- c(10500, 11000, 11500, 12000)

In [46]:
# valvula de seguridad anti atajos
if( PARAM$train_final$APO * PARAM$train_final$ksemillerio  < 100
  | PARAM$train_final$APO < 5
)  {
  # stop("No quiera overfitear")
  cat("No quiera overfitear")
}

In [47]:
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$ksemillerio*PARAM$train_final$APO )]
PARAM$train_final$semillas

[1] 974411 507673 696271 281233 677309 180647 251519 235241 176213 162601
 [11] 138637 519551 807931 471301 152063 823663 874337 466201 206191 787939
 [21] 361327 320141 459847 246833 208799 381371 868327 745187 297371 244147
 [31] 775739 486043 903163 228989 714223 766943 838157 492253 954757 698393
 [41] 424103 964637 581239 315739 868379 589439 114487 645131 794953 356567
 [51] 609821 660799 957059 495629 227627 647161 499327 671837 810239 776219
 [61] 205663 681781 766739 671633 450601 204583 822839 217411 333071 823033
 [71] 631471 595073 139663 564301 459749 713863 979171 119569 935303 440641
 [81] 207187 907811 552263 281839 990281 326707 346739 451783 523771 377129
 [91] 783613 720901 268843 349409 352619 473353 847601 859787 427241 365201

In [48]:
# dejo los datos en formato LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 437055 columnas 1241 


[1] "2025-11-07 21:08:58 EST"

In [49]:
# genero los modelitos
dir.create( "modelitos", showWarnings= FALSE)

param_completo <- copy( PARAM$train_final$param_mejores)

for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
    gc()
  }
}

Sys.time()

[1] "2025-11-07 21:08:58 EST"

Se hace el predict() del modelo en los datos del futuro

In [50]:
# dataset de future, donde en este caso estoy haciendo testing
dfuture <- dataset[foto_mes %in% PARAM$train_final$future ]
mfuture <- data.matrix(dfuture[, campos_buenos, with= FALSE])

dfuture[, ganancia := ifelse(clase_ternaria=="BAJA+2", 780000, -20000)]

In [51]:
mganancias <- matrix( nrow=PARAM$train_final$APO, ncol= length(PARAM$train_final$cortes) )

if( file.exists("prediccion.txt") )
  file.remove("prediccion.txt")

In [52]:
# aplico el modelo a los datos del future

for( vapo in seq(PARAM$train_final$APO) ) {
  # inicializacion en CERO
  vpred_acum <- rep(0.0, nrow(dfuture))
  qacumulados <- 0

  desde <- 1 + (vapo-1)*PARAM$train_final$ksemillerio
  hasta <- desde + PARAM$train_final$ksemillerio - 1
  semillas <- PARAM$train_final$semillas[desde:hasta]

  for( sem in semillas ) {

    arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
    if( file.exists( arch_modelo ) )
    {
      modelo_final <- lgb.load(arch_modelo) # leo del disco
      # hago el predict() y acumulo
      vpred_acum <- vpred_acum + predict(modelo_final, mfuture)
      qacumulados <- qacumulados + 1
      rm(modelo_final)
      gc()
    }
  }

  if( qacumulados > 0 ) {
    vpred_acum <- vpred_acum / qacumulados  # paso a probabildiad
    # tabla de prediccion, puede ser util para futuros ensembles
    #  ya que le modelo ganador va a ser un ensemble de LightGBMs

    tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes, ganancia)]
    tb_prediccion[, meta_modelo := vapo]
    tb_prediccion[, prob := vpred_acum ]
    setorder( tb_prediccion, -prob )
    tb_prediccion[, gan_acum := cumsum(ganancia)]
    tb_prediccion[, ganancia := NULL ]

    # acumulo las ganancias
    for( icor in seq(length(PARAM$train_final$cortes)) ){
      mganancias[ vapo, icor ] <- tb_prediccion[ PARAM$train_final$cortes[icor], gan_acum ]
    }

    # grabo las probabilidades del modelo
    fwrite(tb_prediccion,
      file= "prediccion.txt",
      sep= "\t",
      append= TRUE
    )

    rm(tb_prediccion)
    gc()
  }
}

Sys.time()

[1] "2025-11-07 21:17:29 EST"

In [53]:
mganancias

407600000,411200000,406800000,404000000
407600000,406400000,408400000,407200000
410800000,408800000,406800000,405600000
407600000,407200000,403600000,403200000
411600000,411200000,407600000,407200000
406000000,402400000,404400000,405600000
407600000,412000000,408400000,408000000
405200000,407200000,409200000,404000000
407600000,405600000,407600000,404800000
408400000,409600000,407600000,403200000


### Clasificacion

Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión 

In [54]:
# genero archivos con los  "envios" mejores
dir.create("kaggle", showWarnings=FALSE)

tb_prediccion <- fread("prediccion.txt")

In [55]:
# genero archivos de fantasia, que NO son el que voy a subir a la Pseudo Competencia Kaggle
envios <- 11000

for( vapo in seq(PARAM$train_final$APO) ) {
  if( tb_prediccion[meta_modelo==vapo, .N] > 0 ) {
    tb_pred <- tb_prediccion[meta_modelo==vapo]
    setorder( tb_pred, -prob )
    tb_pred[, Predicted := 0L] # seteo inicial a 0
    tb_pred[1:envios, Predicted := 1L] # marco los primeros

    archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", vapo, "_", envios, ".csv")

    # grabo el archivo
    fwrite(tb_pred[, list(numero_de_cliente, Predicted)],
      file= archivo_kaggle,
      sep= ","
    )

    rm( tb_pred )
    gc()
  }
}

Sys.time()

[1] "2025-11-07 21:17:31 EST"

### Subida a Pseudo Kaggle

Aqui viene la verdadera magia de  APO = A Prueba Overfiteres
<br>Se sube un submit a la Pseudo Competencia Kaggle cuya ganancia coincide casi exactamente con la ganancia MEDIA , promediada  APO veces

In [56]:
colmedias <- colMeans( mganancias, na.rm=TRUE )
mcorte_mejor <- max(colmedias, na.rm=TRUE)
icorte_mejor <- which.max( colmedias )
corte_mejor <- PARAM$train_final$cortes[icorte_mejor]

In [57]:
tbl <- as.data.table( as.list( colmedias ) )
colnames(tbl) <- paste0( "e", PARAM$train_final$cortes )
tbl[, experimento := PARAM$experimento ]

exp_gral <- "/content/buckets/b1/exp/apo-gral"
dir.create(exp_gral, showWarnings=FALSE)
fwrite( tbl,
  file= paste0( exp_gral, "/tb_experimentos.txt"),
  sep= "\t",
  append= TRUE
)

In [58]:
colnames( mganancias ) <- paste0( "e", PARAM$train_final$cortes )
tbl_local <- as.data.table( mganancias )

fwrite( tbl_local,
  file= "tb_apo.txt",
  sep= "\t"
)

In [59]:
icerca <- which.min(  abs( tb_prediccion$gan_acum - mcorte_mejor ) )
vmodelo <- tb_prediccion[ icerca, meta_modelo ]
tb_pred <- tb_prediccion[meta_modelo==vmodelo]

mcorte_mejor
icerca
tb_prediccion[ icerca]

[1] 408160000

[1] 10032

numero_de_cliente,foto_mes,meta_modelo,prob,gan_acum
<int>,<int>,<int>,<dbl>,<dbl>
1209477280,202106,1,0.113957,408160000


In [60]:
icerca <- which.min(  abs( tb_pred$gan_acum - mcorte_mejor ) )
icerca

[1] 10032

In [61]:
icerca <- which.min(  abs( tb_prediccion$gan_acum - mcorte_mejor ) )
vmodelo <- tb_prediccion[ icerca, meta_modelo ]
tb_pred <- tb_prediccion[meta_modelo==vmodelo]

icerca <- which.min(  abs( tb_pred$gan_acum - mcorte_mejor ) )
tb_pred[, Predicted := 0L] # seteo inicial a 0
tb_pred[1:icerca, Predicted := 1L] # marco los primeros

archivo_pseudo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_",  icerca, ".csv")

# grabo el archivo
fwrite(tb_pred[, list(numero_de_cliente, Predicted)],
  file= archivo_pseudo_kaggle,
  sep= ","
)


In [62]:
# la subida a Kaggle
comando <- "kaggle competitions submit"
competencia <- "-c  test-202106"
arch <- paste( "-f", archivo_pseudo_kaggle)
mensaje <-  paste0( "-m 'exp=", PARAM$experimento,
  " envios=", icorte_mejor,"'")

                    
linea <- paste( comando, competencia, arch, mensaje)
salida <- system(linea, intern=TRUE)
cat(salida)

Successfully submitted to Test202106

In [63]:
Sys.time()

[1] "2025-11-07 21:17:35 EST"

## Produccion ZLGBM

Las decisiones que se toman para la construccion del modelo final son:
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en todos los meses salvo los de pandemia esta es una meticulosa decisión.
* Se hace un muy consevador undersampling del **0.50** = 50% de la clase mayoritaria (los "CONTINUA" )
* Obviamente los datos donde se aplica el modelo es el mes  {202106}
* Por experimentos en meses anteriores, se decide cortar en los 11000 registros con mayor probabildiad de POS={"BAJA+1", "BAJA+2"}, , esta es una *enorme* decisión.
* No se optmizan los hiperparámetros de LightGBM, sino que se llama a la libreria *zLightGBM*
* Para *zLightGBM*  se crean **100** canaritos, esta es una decisión enorme !

In [33]:
source("/home/vickydiliscia/dmeyf2025/src/churn/C2010/config/_index.R")

In [37]:
PARAM$train_final

$future
[1] 202106

$meses
 [1] 201901 201902 201903 201904 201905 201906 201907 201908 201909 201910
[11] 201911 201912 202001 202002 202003 202001 202002 202003 202007 202008
[21] 202009 202010 202011 202012 202101 202102 202103 202104

$undersampling
[1] 0.1

In [ ]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$meses]

In [ ]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Final Model

In [ ]:
# utilizo  zLightGBM  la nueva libreria
if( !require("zlightgbm") ) install.packages("https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/zlightgbm_4.6.0.99.tar.gz", repos= NULL, type= "source")
require("zlightgbm")
Sys.time()

In [ ]:
cols0 <- copy(colnames(dataset_train_final))
filas <- nrow(dataset_train_final)

for( i in seq(PARAM$lgbm$canaritos) ){
  dataset_train_final[, paste0("canarito_",i) := runif( filas) ]
}

# las columnas canaritos mandatoriamente van al comienzo del dataset
cols_canaritos <- copy( setdiff( colnames(dataset_train_final), cols0 ) )
setcolorder( dataset_train_final, c( cols_canaritos, cols0 ) )

Sys.time()

In [ ]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train_final),
  c("clase_ternaria", "clase01", "training")
)

In [28]:
campos_buenos

[1] "canarito_1"                                 
  [2] "canarito_2"                                 
  [3] "canarito_3"                                 
  [4] "canarito_4"                                 
  [5] "canarito_5"                                 
  [6] "numero_de_cliente"                          
  [7] "foto_mes"                                   
  [8] "active_quarter"                             
  [9] "cliente_vip"                                
 [10] "internet"                                   
 [11] "cliente_edad"                               
 [12] "cliente_antiguedad"                         
 [13] "mrentabilidad"                              
 [14] "mrentabilidad_annual"                       
 [15] "mcomisiones"                                
 [16] "mactivos_margen"                            
 [17] "mpasivos_margen"                            
 [18] "cproductos"                                 
 [19] "tcuentas"                                   
 [20] "ccuenta_corriente"                          
 [21] "mcuenta_corriente_adicional"                
 [22] "mcuenta_corriente"                          
 [23] "ccaja_ahorro"                               
 [24] "mcaja_ahorro"                               
 [25] "mcaja_ahorro_adicional"                     
 [26] "mcaja_ahorro_dolares"                       
 [27] "cdescubierto_preacordado"                   
 [28] "mcuentas_saldo"                             
 [29] "ctarjeta_debito"                            
 [30] "ctarjeta_debito_transacciones"              
 [31] "mautoservicio"                              
 [32] "ctarjeta_visa"                              
 [33] "ctarjeta_visa_transacciones"                
 [34] "mtarjeta_visa_consumo"                      
 [35] "ctarjeta_master"                            
 [36] "ctarjeta_master_transacciones"              
 [37] "mtarjeta_master_consumo"                    
 [38] "cprestamos_prendarios"                      
 [39] "mprestamos_prendarios"                      
 [40] "cprestamos_hipotecarios"                    
 [41] "mprestamos_hipotecarios"                    
 [42] "cplazo_fijo"                                
 [43] "mplazo_fijo_dolares"                        
 [44] "mplazo_fijo_pesos"                          
 [45] "cinversion1"                                
 [46] "minversion1_pesos"                          
 [47] "minversion1_dolares"                        
 [48] "cinversion2"                                
 [49] "minversion2"                                
 [50] "cseguro_vida"                               
 [51] "cseguro_auto"                               
 [52] "cseguro_vivienda"                           
 [53] "cseguro_accidentes_personales"              
 [54] "ccaja_seguridad"                            
 [55] "cpayroll_trx"                               
 [56] "mpayroll"                                   
 [57] "mpayroll2"                                  
 [58] "cpayroll2_trx"                              
 [59] "ccuenta_debitos_automaticos"                
 [60] "mcuenta_debitos_automaticos"                
 [61] "ctarjeta_visa_debitos_automaticos"          
 [62] "mttarjeta_visa_debitos_automaticos"         
 [63] "ctarjeta_master_debitos_automaticos"        
 [64] "mttarjeta_master_debitos_automaticos"       
 [65] "cpagodeservicios"                           
 [66] "mpagodeservicios"                           
 [67] "cpagomiscuentas"                            
 [68] "mpagomiscuentas"                            
 [69] "ccajeros_propios_descuentos"                
 [70] "mcajeros_propios_descuentos"                
 [71] "ctarjeta_visa_descuentos"                   
 [72] "mtarjeta_visa_descuentos"                   
 [73] "ctarjeta_master_descuentos"                 
 [74] "mtarjeta_master_descuentos"                 
 [75] "ccomisiones_mantenimiento"                  
 [76] "mcomisiones_mantenimiento"                  
 [77] "ccomisiones_otras"                         

In [27]:
# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 389967 columnas 877 


[1] "2025-11-14 17:46:02 UTC"

Nuevos hiperparámetros en zLightGBM

canaritos , entero >=0, cantidad de atributos del dataset que se consideran como canaritos, mandatoriamente en la version actual deben estar al comienzo.
gradient_bound , numero real, >=0 cota que se pone a los scores de las hojas de los arboles, es un learning_rate adaptativo
En el caso que canaritos==0 y gradient_bound==0 entonces xLightGBM se comporta exactamente igual a LightGBM

Tener en cuenta que el hiperparámetro min_data_in_leaf se está dejando en el valor de 20 que es el default de LightGBM
realmente valdria la pena experimentar con distintos valores de min_data_in_leaf, hay potencial de mejora !

####  Entrenamiento del modelo

Aqui se ejecuta zLightGBM
no se utilizan hiperparámetros optimos para controlar el overfitting
zLightGBM sabe cuando no hacer un split
si al hacer el n-simo arbol, no puede hacer el split de la raiz, detiene el crecimiento del ensemble y termina

claramente el min_data_in_leaf=20 que es el default de LightGBM esta jugando un papel importante

In [ ]:
# entreno el modelo

modelo_final <- lgb.train(
  data= dtrain_final,
  param= PARAM$lgbm
)

Sys.time()

In [ ]:
# grabo el modelo generado, esto pude ser levantado por LighGBM en cualquier maquina
lgb.save(modelo_final, file="zmodelo.txt")

# grabo un dataset que tiene el detalle de los arboles de LightGBM
tb_arboles <- lgb.model.dt.tree(modelo_final)
fwrite(tb_arboles, file="tb_arboles.txt", sep="\t")

cat("cantidad arbolitos=", tb_arboles[, max(tree_index)+1],"\n" )
cat("summary de las hojas de los arboles")
summary( tb_arboles[, list(hojas=max(leaf_index, na.rm=TRUE)+1), tree_index][,hojas])

Sys.time()

### Scoring

In [ ]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes %in% PARAM$future]

# penosamente, en la versión actual de zLightGBM  los campos canaritos
#  aunque no se utilizan para nada, también deben estar en el dataset donde se hace el predict()
filas <- nrow(dfuture)

for( i in seq(PARAM$lgbm$canaritos) ){
  dfuture[, paste0("canarito_",i) := runif( filas) ]
}

prediccion <- predict(
  modelo_final,
  data.matrix(dfuture[, campos_buenos, with= FALSE]),
)

In [ ]:
# tabla de prediccion, puede ser util para futuros ensembles
#  ya que le modelo ganador va a ser un ensemble de LightGBMs

tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

### Clasificacion

In [ ]:
Se tomó la decisión de enviar a los 11000 registros con mayor probabilidad de POS={"BAJA+1","BAJA+"}
<br> esto se determinó en forma artesanal analizando meses anterior
<br> esta es una muy importante decisión 

In [ ]:
dir.create("kaggle", showWarnings=FALSE)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

envios <- 11000
tb_prediccion[, Predicted := 0L] # seteo inicial a 0
tb_prediccion[1:envios, Predicted := 1L] # marco los primeros

archivo_kaggle <- paste0(PARAM$paths$kaggle, PARAM$experimento, "_", envios, ".csv")

# grabo el archivo
fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
  file= archivo_kaggle,
  sep= ","
)